In [2]:
import numpy as np
import json
from tensorflow import keras

In [5]:
def get_word_index(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

In [4]:
data = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=10000) 

In [5]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
word_index = data.get_word_index() 

word_index = {k:(v+3) for k, v in word_index.items()} 
word_index["<PAD>"] = 0 
word_index["<START>"] = 1 
word_index["<UNK>"] = 2 
word_index["<UNUSED>"] = 3 

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) 

In [9]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

In [10]:
def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

In [11]:

print(decode_review(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [12]:
vocab_size = 10000
embedding_dim = 16
learning_rate = 0.01

In [23]:
def embed(X,E):
    return np.array([E[i] for i in X])

In [13]:
def ReLU(X):
    return np.maximum(0, X)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [14]:
def init_params():
    E = np.random.randn(vocab_size, embedding_dim)
    W1 = np.random.randn(16,16)
    b1 = np.random.randn(16,)
    W2 = np.random.randn(16,1)
    b2 = np.random.randn(1,)
    return E, W1, b1, W2, b2

In [15]:
def forward_prop(E, W1, b1, W2, b2, X):
    Z0 = embed(X,E)
    A1 = Z0
    Z1 = np.mean(A1, axis=0)
    A2 = Z1
    Z2 = A2.dot(W1) + b1
    A3 = ReLU(Z2)
    Z3 = A3.dot(W2) + b2
    A4 = sigmoid(Z3)
    return A1, A2, Z2, A3, Z3, A4

In [16]:
def back_prop(A1, A2, Z2, A3, A4, Y, E, W1, W2, X):
    m=Y.size
    dZ3 = (A4 - Y)/m
    dW2 = A3.reshape(-1, 1) * dZ3
    db2 = dZ3.flatten()

    dA3 = dZ3 * W2.flatten()
    dZ2 = dA3 * (Z2 > 0)
    dW1 = A2.reshape(-1, 1).dot(dZ2.reshape(1, -1))
    db1 = dZ2.flatten()

    dA2 = dZ2.dot(W1.T)
    dZ1 = dA2 / A1.shape[0]
    dA1 = np.tile(dZ1, (A1.shape[0], 1))

    dE = np.zeros_like(E)
    for t, tok in enumerate(X):
        dE[tok] += dA1[t]
    
    return dE, dW1, db1, dW2, db2

In [17]:
def update_params(E, W1, b1, W2, b2, dE, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    E  = E - learning_rate * dE
    return E, W1, b1, W2, b2

In [18]:
def get_predictions(A4):
    return np.argmax(A4, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions==Y)/Y.size

In [ ]:
def gradient_descent(X, Y, iterations, learning_rate):
    E, W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        correct = 0
        for j in range(len(X)):
            A1, A2, Z2, A3, Z3, A4 = forward_prop(E, W1, b1, W2, b2, X[j])
            dE, dW1, db1, dW2, db2 = back_prop(A1, A2, Z2, A3, A4, Y[j], E, W1, W2, X[j])
            E, W1, b1, W2, b2 = update_params(E, W1, b1, W2, b2, dE, dW1, db1, dW2, db2, learning_rate)

            prediction = 1 if A4[0] > 0.5 else 0
            if prediction == Y[j]:
                correct += 1
                
        if i % 10 == 0:
            accuracy = correct / len(X)
            print("Iteration:", i, "Accuracy: ", accuracy)
    return E, W1, b1, W2, b2

In [24]:

E, W1, b1, W2, b2 = gradient_descent(train_data, train_labels, 100, learning_rate)

Iteration: 0 Accuracy:  0.56052
Iteration: 10 Accuracy:  0.75048
Iteration: 10 Accuracy:  0.75048
Iteration: 20 Accuracy:  0.7992
Iteration: 20 Accuracy:  0.7992
Iteration: 30 Accuracy:  0.82392
Iteration: 30 Accuracy:  0.82392
Iteration: 40 Accuracy:  0.8396
Iteration: 40 Accuracy:  0.8396
Iteration: 50 Accuracy:  0.85136
Iteration: 50 Accuracy:  0.85136
Iteration: 60 Accuracy:  0.85968
Iteration: 60 Accuracy:  0.85968
Iteration: 70 Accuracy:  0.86908
Iteration: 70 Accuracy:  0.86908
Iteration: 80 Accuracy:  0.87464
Iteration: 80 Accuracy:  0.87464
Iteration: 90 Accuracy:  0.8812
Iteration: 90 Accuracy:  0.8812


In [ ]:
def test_model(E, W1, b1, W2, b2, X_test, Y_test):
    correct = 0
    for i in range(len(X_test)):
        A1, A2, Z2, A3, Z3, A4 = forward_prop(E, W1, b1, W2, b2, X_test[i])
        prediction = 1 if A4[0] > 0.5 else 0
        if prediction == Y_test[i]:
            correct += 1
    
    accuracy = correct / len(X_test)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

In [228]:
test_model(E, W1, b1, W2, b2, test_data,test_labels)

0.3082849029847548
7
Test Accuracy: 0.0002


0.00024

In [262]:
np.savez("my_model.npz",
         E=E,
         W1=W1,
         b1=b1,
         W2=W2,
         b2=b2)